In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import estnltk
from estnltk import Text

SyntaxError: invalid syntax (text.py, line 14)

In [4]:
from estnltk.taggers.estner.refac.ner import NerTagger
ntager = NerTagger()


In [3]:
textobject2 = Text("Eesti Vabariik on riik Põhja-Euroopas. Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.")

In [6]:
from estnltk.converters import text_to_dict

text_to_dict(textobject2)

{'layers': [],
 'meta': {},
 'text': 'Eesti Vabariik on riik Põhja-Euroopas. Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.'}

In [4]:
from estnltk.taggers.estner.ner_tagger import NerTagger as nt2



In [5]:
nertagger = nt2()

In [6]:
nertagger.tag(textobject2)

Text(text='Eesti Vabariik on riik Põhja-Euroopas. Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.')

In [7]:
textobject2.ner

Layer(name='ner', attributes=('nertag', 'name'), spans=SL[EnvelopingSpan(['Eesti', 'Vabariik'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Põhja-Euroopas'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Eesti'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Soome', 'lahe', 'Soome', 'Vabariigiga'], [{'nertag': 'LOC', 'name': 'po'}])])

In [8]:
class BaseFeatureExtractor(object):
    """Base class for all feature extractors."""

    def __init__(self, *args, **kwargs):
        pass

    def prepare(self, docs):
        ''' Called before feature extraction actually happens. Can be used to 
        collect global statistics on the corpus. '''
        pass

    def process(self, doc):
        for token in doc.tokens:
            self._process(token)

    def _process(self, token):
        raise NotImplementedError("Not implemented!")


class MorphFeatureExtractor(BaseFeatureExtractor):
    """Extracts features provided by the morphological analyser pyvabamorf. """

    def _process(self, t):
        LEM = "lem"
        POS = "pos"
        PROP = "prop"
        PREF = "pref"
        POST = "post"
        CASE = "case"
        END = "end"
        PUN = "pun"
        t[LEM] = get_lemma(t.lemma)
        t[POS] = get_pos(t.morph)
        t[PROP] = b(is_prop(t.morph))
        t[PREF], t[POST] = get_word_parts(t.lemma)
        t[CASE] = get_case(t.morph)
        t[END] = get_ending(t.lemma)
        t[PUN] = b(t[POS] == '_Z_')

In [16]:
def get_shape(token):
    r = u''
    for c in token:
        if c.isupper():
            r += 'U'
        elif c.islower():
            r += 'L'
        elif c.isdigit():
            r += 'D'
        elif c in ('.', ','):
            r += '.'
        elif c in (';', ':', '?', '!'):
            r += ';'
        elif c in ('+', '-', '*', '/', '=', '|', '_'):
            r += '-'
        elif c in ('(', '{', '[', '<'):
            r += '('
        elif c in (')', '}', ']', '>'):
            r += ')'
        else:
            r += c
    return r


def degenerate(src):
    dst = u''
    for c in src:
        if not dst or dst[-1] != c:
            dst += c
    return dst


def get_2d(token):
    return len(token) == 2 and token.isdigit()


def get_4d(token):
    return len(token) == 4 and token.isdigit()


def get_da(token):
    bd = False
    ba = False
    for c in token:
        if c.isdigit():
            bd = True
        elif c.isalpha():
            ba = True
        else:
            return False
    return bd and ba


def get_dand(token, p):
    bd = False
    bdd = False
    for c in token:
        if c.isdigit():
            bd = True
        elif c == p:
            bdd = True
        else:
            return False
    return bd and bdd


def get_all_other(token):
    for c in token:
        if c.isalnum():
            return False
    return True


def get_capperiod(token):
    return len(token) == 2 and token[0].isupper() and token[1] == '.'


def contains_upper(token):
    b = False
    for c in token:
        b |= c.isupper()
    return b


def contains_lower(token):
    b = False
    for c in token:
        b |= c.islower()
    return b


def contains_alpha(token):
    b = False
    for c in token:
        b |= c.isalpha()
    return b


def contains_digit(token):
    b = False
    for c in token:
        b |= c.isdigit()
    return b


def contains_symbol(token):
    for c in token:
        if not c.isalnum():
            return True
    return False


def split_char(token, char):
    return token.split(char, 1) if token.find(char) > -1 else [None, None]


def b(v):
    return 'y' if v else None


# morphological features
def get_lemma(morph_lemma):
    if len(morph_lemma) > 1:
        ridx = morph_lemma.rfind("=")
        if ridx == -1: ridx = morph_lemma.rfind("+")
        if ridx == -1: ridx = len(morph_lemma)
        if ridx > 0:
            morph_lemma = morph_lemma[:ridx]
        lemma = re.sub(r'(?<=[^\W_])_(?=[^\W_])', '', morph_lemma)
    else:
        lemma = morph_lemma
    return lemma.lower()


def get_word_parts(morph_lemma):
    ridx = morph_lemma.rfind("=")
    if ridx == -1: ridx = morph_lemma.rfind("+")
    if ridx == -1: ridx = len(morph_lemma)
    lemma = morph_lemma[:ridx]
    lemma = lemma.lower()
    chunks = lemma.split("_")
    if len(chunks) > 1:
        prefix, postfix = chunks[0], chunks[-1]
    else:
        prefix, postfix = None, None
    return prefix, postfix


def get_case(morph):
    if morph.startswith("_S_") or morph.startswith("_H_"):
        try:
            case = morph[morph.rindex(" ") + 1: len(morph)]
        except ValueError:
            case = None
    else:
        case = None
    return case


def get_ending(morph_lemma):
    try:
        end = morph_lemma[morph_lemma.index("+") + 1:]
    except ValueError:
        return None
    if end == "0":
        return None
    else:
        return end


def get_pos(morph):
    pos = morph.split()[0]
    return pos


def is_prop(morph):
    return morph.split()[0] == "_H_"

In [1]:
mfe = MorphFeatureExtractor()

NameError: name 'MorphFeatureExtractor' is not defined

In [12]:
from estnltk.taggers.estner.refac.ner import json_document_to_estner_document



In [56]:
nerdoc = json_document_to_estner_document(textobject2)

In [18]:
import re

In [34]:
from copy import deepcopy

In [59]:
textobject2.tag_layer()

Text(text='Eesti Vabariik on riik Põhja-Euroopas. Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.')

In [63]:
textobject2.morph_analysis[0]

text,normalized_text,lemma,root,root_tokens,ending,clitic,form,partofspeech
Eesti,Eesti,Eesti,Eesti,['Eesti'],0,,sg g,H


In [67]:
for token in nerdoc.tokens:
    token2 = deepcopy(token)
    mfe._process(token2)
    print(token2)
    print(token2.feature_list())

{'lem': 'eesti', 'pos': '_H_', 'prop': 'y', 'case': 'g'}


KeyError: 'F'

In [27]:
mfe._process(nerdoc.tokens[0])

TypeError: 'generator' object is not subscriptable

In [22]:
nerdoc

Eesti Vabariik on riik Põhja-Euroopas . Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga .

In [9]:
sisend = [{'named_entities': [{'end': 14, 'label': 'LOC', 'start': 0},
   {'end': 37, 'label': 'LOC', 'start': 23}],
  'paragraphs': [{'end': 43, 'start': 0}],
  'sentences': [{'end': 38, 'start': 0}],
  'text': 'Eesti Vabariik on riik Põhja-Euroopas.\n    ',
  'words': [{'analysis': [{'clitic': '',
      'ending': '0',
      'form': 'sg g',
      'lemma': 'Eesti',
      'partofspeech': 'H',
      'root': 'Eesti',
      'root_tokens': ['Eesti']}],
    'end': 5,
    'label': 'B-LOC',
    'start': 0,
    'text': 'Eesti'},
   {'analysis': [{'clitic': '',
      'ending': '0',
      'form': 'sg n',
      'lemma': 'vabariik',
      'partofspeech': 'S',
      'root': 'vaba_riik',
      'root_tokens': ['vaba', 'riik']}],
    'end': 14,
    'label': 'I-LOC',
    'start': 6,
    'text': 'Vabariik'},
   {'analysis': [{'clitic': '',
      'ending': '0',
      'form': 'b',
      'lemma': 'olema',
      'partofspeech': 'V',
      'root': 'ole',
      'root_tokens': ['ole']},
     {'clitic': '',
      'ending': '0',
      'form': 'vad',
      'lemma': 'olema',
      'partofspeech': 'V',
      'root': 'ole',
      'root_tokens': ['ole']}],
    'end': 17,
    'label': 'O',
    'start': 15,
    'text': 'on'},
   {'analysis': [{'clitic': '',
      'ending': '0',
      'form': 'sg n',
      'lemma': 'riik',
      'partofspeech': 'S',
      'root': 'riik',
      'root_tokens': ['riik']}],
    'end': 22,
    'label': 'O',
    'start': 18,
    'text': 'riik'},
   {'analysis': [{'clitic': '',
      'ending': 's',
      'form': 'sg in',
      'lemma': 'Põhja-Euroobas',
      'partofspeech': 'H',
      'root': 'Põhja-Eu_roobas',
      'root_tokens': ['Põhja', 'Eu', 'roobas']},
     {'clitic': '',
      'ending': 's',
      'form': 'sg in',
      'lemma': 'Põhja-Euroopa',
      'partofspeech': 'H',
      'root': 'Põhja-Euroopa',
      'root_tokens': ['Põhja', 'Euroopa']}],
    'end': 37,
    'label': 'B-LOC',
    'start': 23,
    'text': 'Põhja-Euroopas'},
   {'analysis': [{'clitic': '',
      'ending': '',
      'form': '',
      'lemma': '.',
      'partofspeech': 'Z',
      'root': '.',
      'root_tokens': ['.']}],
    'end': 38,
    'label': 'O',
    'start': 37,
    'text': '.'}]}]

In [19]:
tagged2 = ntager.tag_documents([textobject2])

In [20]:
tagged2[0].ner

Layer(name='ner', attributes=('nertag', 'name'), spans=SL[EnvelopingSpan(['Eesti', 'Vabariik'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Põhja-Euroopas'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Eesti'], [{'nertag': 'LOC', 'name': 'po'}]),
EnvelopingSpan(['Soome', 'lahe', 'Soome', 'Vabariigiga'], [{'nertag': 'LOC', 'name': 'po'}])])

In [25]:
for i, tekst in enumerate(textobject.words):
    print(tekst)
    break

Span('Eesti', [{'normalized_form': None}])


In [16]:
tagged[0].ner[0].nertag

['B-LOC',
 'I-LOC',
 'O',
 'O',
 'B-LOC',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'B-LOC',
 'I-LOC',
 'B-LOC',
 'I-LOC',
 'O']

In [15]:
textobject

Text(text='Eesti Vabariik on riik Põhja-Euroopas. Eesti piirneb põhjas üle Soome lahe Soome Vabariigiga.')